## Segmenting and Clustering Neighborhoods in Toronto

### 1. Importing Libraries

In [210]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

### 2. Scraping table using BeautifulSoup

In [185]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
req=requests.get(url)
soup=BeautifulSoup(req.content, 'html.parser')
table = soup.find(name='table')

### 3. Transform html table in DataFrame and pre-processing it

In [186]:
#Create a DataFrame with table html
df=pd.read_html(str(table))[0]

#Exclude 'Not assigned' Borough
df=df[df['Borough']!='Not assigned']

#Reset index after filter
df=df.reset_index(drop=True)

df.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [187]:
df.shape

(103, 3)

### 4. Get Latitude and Logitude values and concatenate with data

In [188]:
#transform CSV in a DataFrame
lati_long=pd.read_csv('https://cocl.us/Geospatial_data')
#Concatenate with Wikipedia DataFrame
df=pd.concat([df,lati_long],axis=1)

In [189]:
df.head(12)

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",M1J,43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills,M1L,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848


In [190]:
df_toronto=df[df['Borough'].str.contains('Toronto')]
df_toronto

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
15,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
19,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975
20,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714
24,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259
25,M6G,Downtown Toronto,Christie,M3A,43.753259,-79.329656
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",M3K,43.737473,-79.464763
31,M6H,West Toronto,"Dufferin, Dovercourt Village",M3L,43.739015,-79.506944


### 5. Get data from Foursquare API

In [191]:
# API Credentials
CLIENT_ID = 'XHJAVHRTTXAZEUQZZSF51VUPRZH1MEVZTMQC020RLTMBGKI1' # your Foursquare ID
CLIENT_SECRET = 'DLAMV1EWXZL4VT1OMGBASUPH2KWZXO23HIXMPNMFQ11HFTWG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [192]:
# Funcion to get Venues in a DataFrame
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [193]:
# Get Toronto_venues in a DataFrame
toronto_venues = getNearbyVenues(names=df_toronto['Neighbourhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [194]:
print(toronto_venues.shape)
toronto_venues.head()

(749, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


### 6. Pre-processing data

In [195]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,Church and Wellesley,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
745,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
746,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
747,"Business reply mail Processing Centre, South C...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [196]:
#Group by mean
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Berczy Park,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
1,"Brockton, Parkdale Village, Exhibition Place",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.054054,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.027027,0.0000,0.00,0.027027
2,"Business reply mail Processing Centre, South C...",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
4,Central Bay Street,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
5,Christie,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
6,Church and Wellesley,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
7,"Commerce Court, Victoria Hotel",0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.500000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
8,Davisville,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.00,0.000000,0.000000,0.0000,0.00,0.000000
9,Davisville North,0.0000,0.0000,0.0000,0.000,0.000,0.0000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.0,0.054054,0.00,0.040541,0.013514,0.0000,0.00,0.000000


### 7. KMeans Algorithm

In [197]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 0, 0, 0, 0, 3, 0, 4, 0, 0], dtype=int32)

### 8. Defining TOP 10 Venues and adding to Data

In [198]:
#Function to return most common venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [199]:
# Choose Top 10 venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts School,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Fish Market,Park,Neighborhood
2,"Business reply mail Processing Centre, South C...",Park,Pizza Place,Sandwich Place,Bus Line,Diner,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Gym / Fitness Center,Skate Park,Garden Center,Garden,Fast Food Restaurant,Farmers Market,Light Rail Station,Comic Shop,Pizza Place,Recording Studio
4,Central Bay Street,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


In [200]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head(10)# check the last columns!

,Postal Code,Borough,Neighbourhood,Postal Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",M1E,43.763573,-79.188711,0,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Intersection,Bank,Restaurant,Rental Car Location,Farmers Market,Falafel Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M1H,43.773136,-79.239476,0,Bank,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Fried Chicken Joint,Donut Shop,Doner Restaurant
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848,0,Skating Rink,College Stadium,General Entertainment,Café,Yoga Studio,Distribution Center,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
15,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,0,Fast Food Restaurant,Grocery Store,Pizza Place,Sandwich Place,Bank,Chinese Restaurant,Coffee Shop,Indian Restaurant,Breakfast Spot,Pharmacy
19,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975,0,Japanese Restaurant,Bank,Café,Chinese Restaurant,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
20,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714,2,Martial Arts School,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
24,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259,0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
25,M6G,Downtown Toronto,Christie,M3A,43.753259,-79.329656,3,Park,Food & Drink Shop,Field,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",M3K,43.737473,-79.464763,3,Airport,Park,Greek Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
31,M6H,West Toronto,"Dufferin, Dovercourt Village",M3L,43.739015,-79.506944,0,Grocery Store,Park,Shopping Mall,Hotel,Bank,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### 9. Create a Folium Map with the clusters for Data Visualization

In [225]:
# create map

map_clusters = folium.Map(location=[43.651571,-79.484450], zoom_start=11) #I choose Regent Park and Harbourfront from Downtown Toronto for start point of visualization

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 10. Showing the clusters DataFrames

In [216]:
#Cluster 1:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,-79.188711,0,Breakfast Spot,Electronics Store,Medical Center,Mexican Restaurant,Intersection,Bank,Restaurant,Rental Car Location,Farmers Market,Falafel Restaurant
4,Downtown Toronto,-79.239476,0,Bank,Hakka Restaurant,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Gas Station,Bakery,Fried Chicken Joint,Donut Shop,Doner Restaurant
9,Downtown Toronto,-79.264848,0,Skating Rink,College Stadium,General Entertainment,Café,Yoga Studio,Distribution Center,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
15,Downtown Toronto,-79.318389,0,Fast Food Restaurant,Grocery Store,Pizza Place,Sandwich Place,Bank,Chinese Restaurant,Coffee Shop,Indian Restaurant,Breakfast Spot,Pharmacy
19,East Toronto,-79.385975,0,Japanese Restaurant,Bank,Café,Chinese Restaurant,Yoga Studio,Dog Run,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store
24,Downtown Toronto,-79.442259,0,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Butcher,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
31,West Toronto,-79.506944,0,Grocery Store,Park,Shopping Mall,Hotel,Bank,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
36,Downtown Toronto,-79.318389,0,Skating Rink,Curling Ice,Beer Store,Athletics & Sports,Dance Studio,Park,Cuban Restaurant,Creperie,Fast Food Restaurant,Farmers Market
37,West Toronto,-79.293031,0,Health Food Store,Trail,Neighborhood,Pub,Yoga Studio,Discount Store,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,East Toronto,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Restaurant,Ice Cream Shop,Furniture / Home Store,Spa,Indian Restaurant,Lounge


In [218]:
#Cluster 2:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
91,Downtown Toronto,-79.498509,1,Construction & Landscaping,Baseball Field,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
97,Downtown Toronto,-79.532242,1,Baseball Field,Yoga Studio,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [219]:
#Cluster 3:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Downtown Toronto,-79.374714,2,Martial Arts School,Yoga Studio,Distribution Center,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [220]:
#Cluster 4:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,Downtown Toronto,-79.329656,3,Park,Food & Drink Shop,Field,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
30,Downtown Toronto,-79.464763,3,Airport,Park,Greek Restaurant,Fast Food Restaurant,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
74,Central Toronto,-79.453512,3,Park,Women's Store,Pool,Discount Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


In [221]:
#Cluster 5:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Downtown Toronto,-79.38316,4,Playground,Trail,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
